In [1]:
#Workshop 15: เกม Shooting Meteors

import sys, pygame, random
from pygame.locals import *

SCREEN_W = 600
SCREEN_H = 400
BLUESKY = (200, 220, 255)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
CYAN = (0, 255, 255)
FPS = 30

pygame.init()
pygame.display.set_caption('Shooting Meteors By Pantawat no.12')
screen = pygame.display.set_mode((SCREEN_W, SCREEN_H))
screen_rect = screen.get_rect()

clock = pygame.time.Clock()
img_path = r'assets/pygame/images'
font_path = r'assets/pygame/fonts'
snd_path = r'assets/pygame/sounds'

win = False
lose = False
explosion_end = False
game_over = False

spaceship_power = 20    #พลังงานเริ่มต้นของยานเท่ากับ 20%

#--------------- Spaceship Sprite ---------------
spaceship_img = pygame.image.load(fr'{img_path}/spaceship.png')

class Spaceship(pygame.sprite.Sprite):
    def __init__(self):
        super(Spaceship, self).__init__()
        self.image = spaceship_img
        self.rect = self.image.get_rect(
            centerx=screen_rect.centerx,
            bottom=(SCREEN_H - 5)
        ) 
        self.speedx = 10

    def update(self, keys):
        #ยานจะเลื่อนได้เฉพาะในแนวซ้าย-ขวา
        if keys[K_RIGHT]: 
            self.rect.move_ip(self.speedx, 0)
            if self.rect.right >  SCREEN_W:
                self.rect.right = SCREEN_W

        elif keys[K_LEFT]: 
            self.rect.move_ip(-self.speedx, 0)
            if self.rect.left < 0:
                self.rect.left = 0           


#--------------- Bullet Sprite ---------------
bullet_img = pygame.image.load(fr'{img_path}/bullet-laser.png')

class Bullet(pygame.sprite.Sprite):
    def __init__(self, pos):
        super(Bullet, self).__init__()
        self.image = bullet_img
        self.rect = self.image.get_rect(midtop=pos)
        self.speed = 30

    def update(self):
        self.rect.move_ip(0, -self.speed)
        if self.rect.bottom < 0:    #ถ้าหลุดจากขอบบน ให้ทำลายทิ้งไป
            self.kill()  


#--------------- Meteor Sprite ---------------
#มี Meteor ให้เลือก 3 ขนาด และในแต่ละขนาดจะออปเป็น 2 รูปแบบ
mtr_lg_1 = pygame.image.load(fr'{img_path}/meteor-lg-1.png')  
mtr_lg_2 = pygame.image.load(fr'{img_path}/meteor-lg-2.png') 
mtr_md_1 = pygame.image.load(fr'{img_path}/meteor-md-1.png') 
mtr_md_2 = pygame.image.load(fr'{img_path}/meteor-md-2.png') 
mtr_sm_1 = pygame.image.load(fr'{img_path}/meteor-sm-1.png') 
mtr_sm_2 = pygame.image.load(fr'{img_path}/meteor-sm-2.png')  
meteors = [mtr_lg_1, mtr_lg_2, mtr_md_1, mtr_md_2, mtr_sm_1, mtr_sm_2]     

class Meteor(pygame.sprite.Sprite):
    def __init__(self):
        super(Meteor, self).__init__()
        r = random.choice(meteors)  
        self.image = r  
        self.speedx = random.randrange(-5, 5)

        #ความเร็วและพลังงานที่อุกกาบาตลูกนั้นปลดปล่อยออกมา 
        #จะขึ้นกับขนาดของอุกกาบาต
        self.power = 0
        i = meteors.index(r)
        if i <= 1:      #ใหญ่
            self.speedy = random.randrange(4, 6) 
            self.power = 3
        elif i <= 3:    #กลาง
            self.speedy = random.randrange(6, 8) 
            self.power = 4
        else:           #เล็ก
            self.speedy = random.randrange(8, 10) 
            self.power = 5

        w = self.image.get_width()
        start_left = random.randint(w, (SCREEN_W - w))
        start_bottom = random.randint(-300, 0)
        self.rect = self.image.get_rect(
            left=start_left, bottom=start_bottom
        )

    def update(self):
        self.rect.move_ip(self.speedx, self.speedy)
        if self.rect.right < 0 or \
            self.rect.left > SCREEN_W or \
            self.rect.top > SCREEN_H:

            self.kill()


#--------------- Explosion Sprite ---------------
exp_img =  pygame.image.load(fr'{img_path}/explosion.png') 
exp_num_rows = 3       
exp_num_cols = 3        
exp_subimg_w = exp_img.get_width() // exp_num_rows  
exp_subimg_h = exp_img.get_height() // exp_num_cols 
exp_subimgs = []        

for r in range(exp_num_rows):
    for c in range(exp_num_cols):
        x = c * exp_subimg_w
        y = r * exp_subimg_h
        img = exp_img.subsurface(x, y, exp_subimg_w, exp_subimg_h)
        exp_subimgs.append(img)

exp_num_subimgs = len(exp_subimgs)          
exp_repeat = FPS // exp_num_subimgs  
exp_last_frame = (exp_repeat * exp_num_subimgs) - 1    

class Explosion(pygame.sprite.Sprite):
    def __init__(self, pos):       
        super(Explosion, self).__init__()
        self.image = exp_subimgs[0]
        left = pos[0] - (exp_subimg_w // 2)
        top = pos[1] - (exp_subimg_h // 2)
        self.rect = pygame.Rect(
            left, top, exp_subimg_w, exp_subimg_w
        )
        self.index = 0

    def update(self):
        global explosion_end
        if self.index >= exp_last_frame:
            self.kill()
            explosion_end = True
        else:
            i = self.index // exp_repeat
            self.image = exp_subimgs[i]
            screen.blit(self.image, self.rect)
            self.index += 1


#--------------- Images and Sounds ---------------
cover = pygame.image.load(fr'{img_path}/galaxy-bg.jpg')
bg_space = pygame.image.load(fr'{img_path}/space.jpg')

intro_snd = pygame.mixer.Sound(fr'{snd_path}/game-intro.wav')
bg_snd = pygame.mixer.Sound(fr'{snd_path}/space-bass.wav')
bullet_snd = pygame.mixer.Sound(fr'{snd_path}/pew.wav')
exp_snd = pygame.mixer.Sound(fr'{snd_path}/explosion.wav')
exp_snd_big = pygame.mixer.Sound(fr'{snd_path}/big-explosion.wav')


#--------------- Text & Intro Screen ---------------
def draw_text(text, size, colour, x, y, fontfile=None):
    #ถ้าไม่ระบุชื่อไฟล์ของฟอนต์ ให้ใช้ฟอนต์ของระบบ
    if fontfile == None:
        font = pygame.font.SysFont(None, size)
    else:
        font = pygame.font.Font(fontfile, size)

    text_surface = font.render(text, True, colour)
    text_rect = text_surface.get_rect(midtop=(x, y))
    screen.blit(text_surface, text_rect)

def intro_screen():
    intro_snd.play(-1)
    screen.fill(BLUE)
    #ภาพหน้าปก ซึ่งจะแสดงเป็นพื้นหลังของหน้าจอเริ่มต้น
    
    screen.blit(cover, cover.get_rect())    #วาดภาพหน้าปก

    #ชื่อเกม ใช้ฟอนต์ที่ชื่อ Meteora
    draw_text("Shooting Meteors", 72, WHITE, 
             screen_rect.centerx, 30, 
             fr'{font_path}/Meteora.ttf'
    )
    draw_text("Use arrow keys to move spaceship", 24, GREEN, 
            screen_rect.centerx, 200)

    draw_text("Use spacebar to shoot meteors", 24, CYAN, 
            screen_rect.centerx, 230)

    #แสดงปุ่ม EXIT
    btn_exit_image = pygame.image.load(fr'{img_path}/btn-exit-2.png')
    btn_exit_rect = btn_exit_image.get_rect(
        right=screen_rect.centerx - 50, top=300
    )

    #แสดงปุ่ม START
    btn_start_image = pygame.image.load(fr'{img_path}/btn-start-2.png')
    btn_start_rect = btn_start_image.get_rect(
        left=screen_rect.centerx + 50, top=300
    )

    screen.blit(btn_start_image, btn_start_rect)
    screen.blit(btn_exit_image, btn_exit_rect)

    pygame.display.flip()

    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == MOUSEBUTTONDOWN:
                #ถ้าคลิกที่ปุ่ม START ให้เริ่มเล่นเกม
                if btn_start_rect.collidepoint(pygame.mouse.get_pos()):
                    intro_snd.stop()
                    waiting = False
                #ถ้าคลิกปุ่ม EXIT ให้ออกจากเกม
                elif btn_exit_rect.collidepoint(pygame.mouse.get_pos()):
                    pygame.quit()
                    sys.exit()    

#---------------------------------------------
playing = False
running = True
while running:
    if not playing:
        intro_screen()

        playing = True

        #เราต้องกำหนดค่าตัวแปรพร้อมอินสแตนซ์ที่นี่
        #เพื่อให้ค่าเริ่มต้นถูกรีเซตทุกครั้งที่จะเริ่มเกมใหม่
        win = False
        lose = False
        explosion_end = False
        game_over = False
        spaceship_power = 20 
                                
        spaceship = Spaceship()
        group_spaceship = pygame.sprite.Group()
        group_spaceship.add(spaceship)

        group_meteor = pygame.sprite.Group()
        group_bullet = pygame.sprite.Group()
        group_explosion = pygame.sprite.Group()

        ADD_METEOR = pygame.USEREVENT + 1
        pygame.time.set_timer(ADD_METEOR, 500)

        bg_snd.play(-1)

    for event in pygame.event.get():
        if event.type == QUIT:
            running = False    
            pygame.quit()
            sys.exit()
        elif event.type == ADD_METEOR and not game_over:
            meteor = Meteor()
            group_meteor.add(meteor)
        
        #ถ้าคลิกเมาส์เมื่อจบเกมแล้ว ให้เข้าสู่หน้าจอเริ่มต้น
        elif event.type == MOUSEBUTTONDOWN and game_over:
            playing = False

        elif event.type == KEYDOWN:
            keys = pygame.key.get_pressed()
            #ถ้ากดปุ่ม <Spacebar> และเหลือพลังงานเพียงพอ ให้ปล่อยกระสุน
            if keys[K_SPACE] and spaceship_power >= 2 and not game_over:       
                pos = spaceship.rect.midtop
                bullet = Bullet(pos)
                group_bullet.add(bullet)
                bullet_snd.play()
                spaceship_power -= 2

    screen.fill(BLACK)
    screen.blit(bg_space, bg_space.get_rect())

    keys = pygame.key.get_pressed()
    spaceship.update(keys)
    group_bullet.update()
    group_meteor.update()
    group_explosion.update()

    group_spaceship.draw(screen)
    group_bullet.draw(screen)
    group_meteor.draw(screen)
    group_explosion.draw(screen)

    #ถ้าจบเกม ให้แสดงข้อความว่าเป็นผู้แพ้หรือชนะ
    if game_over: 
        bg_snd.stop()   #หยุดเล่นเสียงเบื้องหลัง  
        if win:         #ถ้าชนะ ให้แสดงข้อความ
            draw_text('You Win', 60, BLUE, screen_rect.centerx, 150)

        elif explosion_end and lose: #ถ้าแสดงภาพระเบิดครบแล้ว ให้แสดงข้อความว่าแพ้
            draw_text('You Lose', 60, RED, screen_rect.centerx, 150)

        #ไม่ว่าจะแพ้หรือชนะ ให้แสดงข้อความต่อไปนี้    
        if explosion_end or win:
            draw_text('Click mouse to continue...', 24, GREEN, 
                        screen_rect.centerx, 300)
    
    #ถ้ายังไม่จบเกม ให้แสดงระดับพลังงานของยานในขณะนั้น
    else:
        text = f'power: {spaceship_power}%'
        draw_text(text, 24, GREEN, 70, 10)


    pygame.display.flip()
    clock.tick(FPS)

    if game_over: continue
    
    #ตรวจสอบการชน (Bullet vs Meteor)
    hits = pygame.sprite.groupcollide(
            group_meteor, group_bullet, True, True, pygame.sprite.collide_mask
    )
    if len(hits) > 0:
        for hit in hits:
            first_hit = hit
            break
        
        center = first_hit.rect.center
        explosion = Explosion(center)
        group_explosion.add(explosion)
        exp_snd.play()

        #เพิ่มพลังงานให้กับยานเท่ากับขนาดพลังงานของอุกกาบาตลูกนั้น
        spaceship_power += hit.power
        if spaceship_power > 100:
            spaceship_power = 100
        
    #ตรวจสอบการชน (Spaceship vs Meteor)
    hits2 = pygame.sprite.groupcollide(
            group_meteor, group_spaceship, True, True, pygame.sprite.collide_mask
    )
    if len(hits2) > 0:
        for hit2 in hits2:
            first_hit2 = hit2
            break
        
        center = first_hit2.rect.center
        explosion = Explosion(center)
        group_explosion.add(explosion)
        exp_snd_big.play()
        game_over = True        #ถ้าอุกกาบาตชนยาน ถือว่าจบเกม
        lose = True
     
    if spaceship_power < 2:      #ถ้าพลังงานของยานต่ำกว่า 2% ถือว่าจบเกม
        game_over = True
        lose = True
        spaceship.kill()
    elif spaceship_power == 100:    #ถ้าพลังงานของยานครบ 100% ถือว่าในเกมนั้น
        game_over = True
        win = True
        spaceship.kill()
    

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


2023-09-28 17:56:03.017 Python[3281:134414] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


SystemExit: 

/Users/pantawatchamrongpan/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

In [ ]:
%pip install pygame

Note: you may need to restart the kernel to use updated packages.
